In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch as tc
import cupy as cp

from model.ffnn import FFNN

tc.autograd.set_grad_enabled(False)
        
if tc.cuda.is_available():
    print("CUDA tersedia!")
else:
    print("CUDA tidak tersedia!")
    

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
tc.set_default_device(device)
print("Using CUDA device:", tc.cuda.get_device_name(device))


# cp.cuda.Device(0).use()




CUDA tersedia!
Using CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype(np.float64)
y = mnist.target.astype(np.int32)


X = X / 255.0
# print(X)
print("Dataset MNIST_784 dimuat: X shape:", X.shape, " y shape:", y.shape)

Dataset MNIST_784 dimuat: X shape: (70000, 784)  y shape: (70000,)


In [3]:
subset_size = 3000
indices = np.random.choice(np.arange(X.shape[0]), subset_size, replace=False)
X_subset = X.iloc[indices]
y_subset = y.iloc[indices]

X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.3, random_state=42)

print("Data subset: Training:", X_train.shape, "Testing:", X_test.shape)

Data subset: Training: (2100, 784) Testing: (900, 784)


In [4]:
def one_hot(labels, num_classes):
    oh = np.zeros((labels.shape[0], num_classes))
    oh[np.arange(labels.shape[0]), labels] = 1.0
    return oh

num_classes = 10
y_train_oh = one_hot(y_train, num_classes)
y_test_oh = one_hot(y_test, num_classes)

In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(64,), max_iter=1000, random_state=42, learning_rate_init=0.001, batch_size=200, verbose=True)

start_time = time.time()
mlp.fit(X_train, y_train)
sklearn_training_time = time.time() - start_time

#prediksi
y_pred_sklearn = mlp.predict(X_test)
accuracy_sklearn = np.mean(y_pred_sklearn == y_test)
print("Akurasi MLPClassifier:", accuracy_sklearn)
print("Waktu training MLPClassifier: {:.2f} detik".format(sklearn_training_time))

Iteration 1, loss = 2.08322432
Iteration 2, loss = 1.41990161
Iteration 3, loss = 0.95173718
Iteration 4, loss = 0.68128098
Iteration 5, loss = 0.53308723
Iteration 6, loss = 0.44529438
Iteration 7, loss = 0.38639747
Iteration 8, loss = 0.34555080
Iteration 9, loss = 0.31382286
Iteration 10, loss = 0.28757295
Iteration 11, loss = 0.26577654
Iteration 12, loss = 0.24855530
Iteration 13, loss = 0.23084160
Iteration 14, loss = 0.21610197
Iteration 15, loss = 0.20370109
Iteration 16, loss = 0.19341115
Iteration 17, loss = 0.18043783
Iteration 18, loss = 0.16960515
Iteration 19, loss = 0.16039193
Iteration 20, loss = 0.15187546
Iteration 21, loss = 0.14345477
Iteration 22, loss = 0.13667879
Iteration 23, loss = 0.13000398
Iteration 24, loss = 0.12246982
Iteration 25, loss = 0.11544705
Iteration 26, loss = 0.10868367
Iteration 27, loss = 0.10406920
Iteration 28, loss = 0.09882032
Iteration 29, loss = 0.09334160
Iteration 30, loss = 0.08847338
Iteration 31, loss = 0.08440600
Iteration 32, los

In [6]:
# X_train = cp.asarray(X_train)
# y_train_oh = cp.asarray(y_train_oh)
# X_test = cp.asarray(X_test)
# y_test = cp.asarray(y_test)

# custom_model = FFNN(
#     [784, 64, 10],
#     activations_list=['relu', 'sigmoid'],
#     loss_function='mse',  
#     weight_init='random_uniform',
#     init_params={'lower': -1, 'upper': 1}
# )

# epochs_custom = 10000  
# batch_size_custom = 200
# lr_custom = 0.001

# start_time = time.time()
# history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=False)
# custom_training_time = time.time() - start_time
# print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# # Prediksi pada data test
# y_pred_custom = custom_model.forward(X_test)
# # y_pred_custom.data adalah NumPy array, sehingga gunakan np.argmax dengan parameter axis
# y_pred_custom_labels = cp.argmax(y_pred_custom, axis=1)
# accuracy_custom = cp.mean(y_pred_custom_labels == y_test)
# print("Akurasi model custom FFNN:", accuracy_custom)

In [7]:


X_train = tc.tensor(np.asarray(X_train), dtype=tc.float32)
X_test = tc.tensor(np.asarray(X_test), dtype=tc.float32)
y_train_oh = tc.tensor(np.asarray(y_train_oh), dtype=tc.float32)
y_test = tc.tensor(np.asarray(y_test), dtype=tc.int64)

print(X_train.get_device())

# Inisialisasi model custom FFNN
custom_model = FFNN(
    [784, 64, 10],
    activations_list=['relu', 'softmax'],
    loss_function='cce',  
    weight_init='random_uniform',
    init_params={'lower': -1, 'upper': 1}
)
epochs_custom = 100000
batch_size_custom = 200
lr_custom = 0.001

start_time = time.time()
history_custom = custom_model.train(X_train, y_train_oh, epochs=epochs_custom, batch_size=batch_size_custom, learning_rate=lr_custom, verbose=True)
custom_training_time = time.time() - start_time
print(f"Model custom selesai training dalam {custom_training_time:.2f} detik.")

# Prediksi 
y_pred_custom = custom_model.forward(X_test)
y_pred_custom_labels = tc.argmax(y_pred_custom.data, dim=1)
accuracy_custom = (y_pred_custom_labels == y_test).float().mean().item()
print("Akurasi model custom FFNN:", accuracy_custom)
#107,32s tc cpu
#158,16s tc gpu
#215,52s cp gpu
#230,48s np cpu

#555.06 detik

0
Epoch 1/100000 - Train Loss: 17.5780
Epoch 2/100000 - Train Loss: 17.3719
Epoch 3/100000 - Train Loss: 17.0949
Epoch 4/100000 - Train Loss: 16.7880
Epoch 5/100000 - Train Loss: 16.4445
Epoch 6/100000 - Train Loss: 16.0948
Epoch 7/100000 - Train Loss: 15.7472
Epoch 8/100000 - Train Loss: 15.4188
Epoch 9/100000 - Train Loss: 15.1068
Epoch 10/100000 - Train Loss: 14.8031
Epoch 11/100000 - Train Loss: 14.5342
Epoch 12/100000 - Train Loss: 14.2871
Epoch 13/100000 - Train Loss: 14.0608
Epoch 14/100000 - Train Loss: 13.8438
Epoch 15/100000 - Train Loss: 13.6379
Epoch 16/100000 - Train Loss: 13.4413
Epoch 17/100000 - Train Loss: 13.2533
Epoch 18/100000 - Train Loss: 13.0681
Epoch 19/100000 - Train Loss: 12.8905
Epoch 20/100000 - Train Loss: 12.7167
Epoch 21/100000 - Train Loss: 12.5511
Epoch 22/100000 - Train Loss: 12.3854
Epoch 23/100000 - Train Loss: 12.2260
Epoch 24/100000 - Train Loss: 12.0753
Epoch 25/100000 - Train Loss: 11.9190
Epoch 26/100000 - Train Loss: 11.7653
Epoch 27/100000 - T

KeyboardInterrupt: 